In [23]:
# imports for the shake of importing

import keras
import tensorflow as tf
from tensorflow import nn 
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.applications import NASNetLarge
from keras import models, layers, optimizers, backend
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# THICC networks ftw?

My GPU is hating me, I really wish I had a RTX card right now >_>

In [25]:
img_height, img_width = 331, 331  # For NASNetLarge
classnames = ["Background", "BMP2", "Buk-M1-2", "T14", "T90", "ZSU23"]
classes = len(classnames)

## Level 1

#### Copied directly from Ugur's code

Made some modifications fo that it would fit better in a notebook

In [26]:
conv_base1 = NASNetLarge(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

model = models.Sequential()
model.add(conv_base1)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 4032)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                129056    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 85,046,072
Trainable params: 84,849,404
Non-trainable params: 196,668
_________________________________________________________________


In [27]:
# freezing the base
conv_base1.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.RMSprop(lr=0.0002, decay=0.0001), 
              metrics=['acc'])

callbacks_list_L1 = [ModelCheckpoint(filepath='./weights', 
                                     save_weights_only=True, 
                                     monitor='val_acc', 
                                     verbose=1, 
                                     save_best_only=True),
                                     ReduceLROnPlateau(monitor='val_acc', 
                                     factor=0.5, 
                                     patience=1, 
                                     verbose=1),
                                     TensorBoard(log_dir='./TensorBoard/L1')]